In [342]:
!pip3 install numpy>=1.20 matplotlib==3.7.5 pandas scikit-learn imbalanced-learn

In [343]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

# Carregamento do dataset
dataset = pd.read_csv(filepath_or_buffer="/home/wyctor/PROJETOS/pad_ufes_20_eda/data/metadata.csv", sep=",")
dataset


,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,...,NaN,NEV,False,False,False,False,False,False,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,5.0,BCC,True,True,False,True,True,True,PAT_46_881_939.png,True
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1545_1867_547.png,False
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,...,5.0,BCC,True,True,False,False,True,True,PAT_684_1302_588.png,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,PAT_1708,3156,NaN,NaN,NaN,NaN,73,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1708_3156_175.png,False
2294,PAT_46,880,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,12.0,BCC,True,True,False,True,False,False,PAT_46_880_140.png,True
2295,PAT_1343,1217,NaN,NaN,NaN,NaN,74,NaN,NaN,NaN,...,NaN,SEK,False,False,False,False,False,False,PAT_1343_1217_404.png,False
2296,PAT_326,690,False,False,POMERANIA,POMERANIA,58,True,FEMALE,True,...,4.0,BCC,True,False,False,False,False,True,PAT_326_690_823.png,True


Remover os dados que são inválidos

In [344]:
dataset=dataset.dropna()
dataset=dataset.drop(columns=['lesion_id', 'patient_id', 'img_id'])
## dataset=dataset.replace(dataset["background_father"]=="BRASIL", "BRAZIL")
dataset
## Descomentar se for usar a moda das variáveis como entrada
#dataset.fillna(dataset.mode().iloc[0], inplace=True)
#dataset.replace([True, False], [1, 0])

,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,cancer_history,has_piped_water,...,diameter_1,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,biopsed
1,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,True,True,...,6.0,5.0,BCC,True,True,False,True,True,True,True
4,False,True,POMERANIA,POMERANIA,79,False,MALE,True,False,False,...,5.0,5.0,BCC,True,True,False,False,True,True,True
6,False,True,GERMANY,ITALY,52,False,FEMALE,False,True,True,...,15.0,10.0,BCC,False,True,False,True,True,True,True
7,False,False,POMERANIA,POMERANIA,74,True,FEMALE,False,False,False,...,15.0,10.0,BCC,True,True,True,False,True,True,True
9,False,True,GERMANY,GERMANY,58,True,FEMALE,True,True,True,...,9.0,7.0,ACK,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2289,True,False,POMERANIA,POMERANIA,66,False,MALE,True,False,True,...,20.0,15.0,ACK,True,False,True,False,True,True,True
2291,False,False,POMERANIA,POMERANIA,41,True,MALE,False,False,False,...,9.0,5.0,BCC,True,UNK,True,UNK,True,True,True
2292,False,False,POMERANIA,POMERANIA,43,True,FEMALE,True,True,False,...,9.0,5.0,SCC,True,UNK,False,UNK,True,True,True
2294,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,True,True,...,13.0,12.0,BCC,True,True,False,True,False,False,True


Separação entre os dados das features e dos labels

Uso do hot-encoder para converter as variáveis categóricas para inteiras

In [345]:
from sklearn.preprocessing import OneHotEncoder

# Seleção das features
dataset_features = dataset.drop(columns=['diagnostic'])  # Features

# Identificar variáveis categóricas
categorical_cols = dataset_features.select_dtypes(include=['object']).columns

# Criar o OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)

# Aplicar o OneHotEncoder às colunas categóricas
categorical_data = ohe.fit_transform(dataset_features[categorical_cols])

# Criar um DataFrame para os dados one-hot codificados
categorical_df = pd.DataFrame(categorical_data, columns=ohe.get_feature_names_out(categorical_cols), index=dataset_features.index)

# Combinar os dados transformados com as demais features
dataset_features = dataset_features.drop(columns=categorical_cols)  # Remove as colunas categóricas originais
dataset_features = pd.concat([dataset_features, categorical_df], axis=1)

dataset_features = dataset_features.replace([True, False], [1, 0])
# Verificar os primeiros valores após a transformação
print(dataset_features)


      age  fitspatrick  diameter_1  diameter_2  biopsed  smoke_False  \
1      55          3.0         6.0         5.0        1          1.0   
4      79          1.0         5.0         5.0        1          1.0   
6      52          3.0        15.0        10.0        1          1.0   
7      74          1.0        15.0        10.0        1          1.0   
9      58          1.0         9.0         7.0        0          1.0   
...   ...          ...         ...         ...      ...          ...   
2289   66          2.0        20.0        15.0        1          0.0   
2291   41          2.0         9.0         5.0        1          1.0   
2292   43          2.0         9.0         5.0        1          1.0   
2294   55          3.0        13.0        12.0        1          1.0   
2296   58          3.0         5.0         4.0        1          1.0   

      smoke_True  drink_False  drink_True  background_father_AUSTRIA  ...  \
1            0.0          1.0         0.0                 

/tmp/ipykernel_4707/1366617519.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset_features = dataset_features.replace([True, False], [1, 0])


In [346]:
from sklearn.preprocessing import LabelEncoder

# Inicialize o LabelEncoder
le = LabelEncoder()

y = dataset['diagnostic']  # label

# Fit e transform os rótulos em X_train e y_train
y_encoded = le.fit_transform(y)
y_encoded

array([1, 1, 1, ..., 4, 1, 1])

# Sem balancear as amostrar 

In [347]:
from collections import Counter

print(Counter(y_encoded))

Counter({np.int64(1): 845, np.int64(0): 273, np.int64(4): 192, np.int64(3): 72, np.int64(2): 52, np.int64(5): 40})


In [348]:

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(dataset_features, y_encoded, test_size=0.2, random_state=42, shuffle=True)

In [349]:
features = X_train.columns
features

Index(['age', 'fitspatrick', 'diameter_1', 'diameter_2', 'biopsed',
       'smoke_False', 'smoke_True', 'drink_False', 'drink_True',
       'background_father_AUSTRIA', 'background_father_BRAZIL',
       'background_father_CZECH', 'background_father_GERMANY',
       'background_father_ISRAEL', 'background_father_ITALY',
       'background_father_NETHERLANDS', 'background_father_POLAND',
       'background_father_POMERANIA', 'background_father_PORTUGAL',
       'background_father_SPAIN', 'background_father_UNK',
       'background_mother_BRAZIL', 'background_mother_FRANCE',
       'background_mother_GERMANY', 'background_mother_ITALY',
       'background_mother_NETHERLANDS', 'background_mother_NORWAY',
       'background_mother_POLAND', 'background_mother_POMERANIA',
       'background_mother_PORTUGAL', 'background_mother_SPAIN',
       'background_mother_UNK', 'pesticide_False', 'pesticide_True',
       'gender_FEMALE', 'gender_MALE', 'skin_cancer_history_False',
       'skin_cancer_hi

Uso do Naive Bayes para verificar o impacto final de cada variável sobre  modelo

Analisando uma feature por vez no treino, e outras sendo adicionadas com o tempo


In [350]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, roc_auc_score
from sklearn.naive_bayes import GaussianNB, BernoulliNB
# Modelo Naive Bayes
model = BernoulliNB()

# Lista para armazenar os resultados
results = []

# Incrementalmente incluir features
for i in range(1, X_train.shape[1]+1):
    print(f"Feature adicionada: {features[i-1]}\n")

    # Obter a quantidade de linhas de treino e teste
    width_train, _ = X_train.shape
    width_test, _ = X_test.shape
    
    # Selecionar as primeiras `i` features (e mantendo as linhas (amostras))
    X_train_subset = np.array(X_train.iloc[:, :i]).reshape(width_train, -1)
    X_test_subset = np.array(X_test.iloc[:, :i]).reshape(width_test, -1)
    
    # Treinar o modelo com as features selecionadas
    model.fit(X_train_subset, y_train)
    
    # Obter as probabilidades de saída para o conjunto de teste
    probas = model.predict_proba(X_test_subset)

    # Predições do modelo
    predictions = model.predict(X_test_subset)
    
    # Calcular métricas (roc_auc_score usa as probabilidades da classe positiva)
    roc_auc = roc_auc_score(y_test, probas, multi_class='ovr', average='macro') if len(np.unique(y_test)) > 2 else roc_auc_score(y_test, probas[:, 1])

    # Armazenar resultados
    results.append({
        'iteration_nº': i,
        'features_used': str(features[i-1]),
        'probabilities': probas,
        'predictions': predictions,
        'true_labels': y_test,
        'accuracy_score': accuracy_score(y_test, predictions),
        'balanced_accuracy_score': balanced_accuracy_score(y_test, predictions),
        'f1_score': f1_score(y_test, predictions, average='weighted'), 
        'precision_score': precision_score(y_test, predictions, average='weighted'),
        'roc_auc_score': roc_auc
    })


/home/wyctor/miniconda3/envs/lab-life/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wyctor/miniconda3/envs/lab-life/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wyctor/miniconda3/envs/lab-life/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

Feature adicionada: age

Feature adicionada: fitspatrick

Feature adicionada: diameter_1

Feature adicionada: diameter_2

Feature adicionada: biopsed

Feature adicionada: smoke_False

Feature adicionada: smoke_True

Feature adicionada: drink_False

Feature adicionada: drink_True

Feature adicionada: background_father_AUSTRIA

Feature adicionada: background_father_BRAZIL

Feature adicionada: background_father_CZECH

Feature adicionada: background_father_GERMANY

Feature adicionada: background_father_ISRAEL

Feature adicionada: background_father_ITALY

Feature adicionada: background_father_NETHERLANDS

Feature adicionada: background_father_POLAND

Feature adicionada: background_father_POMERANIA

Feature adicionada: background_father_PORTUGAL

Feature adicionada: background_father_SPAIN

Feature adicionada: background_father_UNK

Feature adicionada: background_mother_BRAZIL

Feature adicionada: background_mother_FRANCE

Feature adicionada: background_mother_GERMANY

Feature adicionada: ba

Exportar os dados em um formato de arquivo csv

In [351]:
import pandas as pd

def save_results_to_csv(results, filename='model_results.csv'):
    # Criação de uma lista para armazenar os dados formatados para o CSV
    rows = []
    
    # Itera sobre os resultados e organiza as métricas e as features usadas
    for result in results:
        row = {
            'iteration_nº': result['iteration_nº'],
            'features_used': result['features_used'],
            'accuracy_score': result['accuracy_score'],
            'balanced_accuracy_score': result['balanced_accuracy_score'],
            'f1_score': result['f1_score'],
            'precision_score': result['precision_score'],
            'roc_auc_score': result['roc_auc_score']
        }
        rows.append(row)
    
    # Convertendo a lista de resultados para um DataFrame
    df = pd.DataFrame(rows)
    
    # Salvando o DataFrame em um arquivo CSV
    df.to_csv(filename, index=False)

In [352]:
save_results_to_csv(results, './results/naive-bayes/model_results_with_no_undersampling.csv')

# Usando o RandomUnderSampling


In [353]:
from imblearn.under_sampling import RandomUnderSampler

# Inicialize o RandomUnderSampler
rus = RandomUnderSampler(random_state=42)

# Aplique a subamostragem
dataset_features_undersampled, y_encoded_undersampled = rus.fit_resample(dataset_features, y_encoded)

# Verifique o número de amostras por classe após o balanceamento
from collections import Counter
print("Distribuição original:", Counter(y_encoded))
print("Distribuição após subamostragem:", Counter(y_encoded_undersampled))


Distribuição original: Counter({np.int64(1): 845, np.int64(0): 273, np.int64(4): 192, np.int64(3): 72, np.int64(2): 52, np.int64(5): 40})
Distribuição após subamostragem: Counter({np.int64(0): 40, np.int64(1): 40, np.int64(2): 40, np.int64(3): 40, np.int64(4): 40, np.int64(5): 40})


In [354]:

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(dataset_features_undersampled, y_encoded_undersampled, test_size=0.2, random_state=42, shuffle=True)

In [355]:
width_train, _ = np.array(X_train).shape
width_train

192

In [356]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, roc_auc_score
from sklearn.naive_bayes import GaussianNB, BernoulliNB
# Modelo Naive Bayes
model = BernoulliNB()

# Lista para armazenar os resultados
results = []

# Incrementalmente incluir features
for i in range(1, X_train.shape[1]+1):
    print(f"Feature adicionada: {features[i-1]}\n")

    # Obter a quantidade de linhas de treino e teste
    width_train, _ = X_train.shape
    width_test, _ = X_test.shape
    
    # Selecionar as primeiras `i` features (e mantendo as linhas (amostras))
    X_train_subset = np.array(X_train.iloc[:, :i]).reshape(width_train, -1)
    X_test_subset = np.array(X_test.iloc[:, :i]).reshape(width_test, -1)
    
    # Treinar o modelo com as features selecionadas
    model.fit(X_train_subset, y_train)
    
    # Obter as probabilidades de saída para o conjunto de teste
    probas = model.predict_proba(X_test_subset)

    # Predições do modelo
    predictions = model.predict(X_test_subset)
    
    # Calcular métricas (roc_auc_score usa as probabilidades da classe positiva)
    roc_auc = roc_auc_score(y_test, probas, multi_class='ovr', average='macro') if len(np.unique(y_test)) > 2 else roc_auc_score(y_test, probas[:, 1])

    # Armazenar resultados
    results.append({
        'iteration_nº': i,
        'features_used': str(features[i-1]),
        'probabilities': probas,
        'predictions': predictions,
        'true_labels': y_test,
        'accuracy_score': accuracy_score(y_test, predictions),
        'balanced_accuracy_score': balanced_accuracy_score(y_test, predictions),
        'f1_score': f1_score(y_test, predictions, average='weighted'), 
        'precision_score': precision_score(y_test, predictions, average='weighted'),
        'roc_auc_score': roc_auc
    })


Feature adicionada: age

Feature adicionada: fitspatrick

Feature adicionada: diameter_1

Feature adicionada: diameter_2

Feature adicionada: biopsed

Feature adicionada: smoke_False

Feature adicionada: smoke_True

Feature adicionada: drink_False

Feature adicionada: drink_True

Feature adicionada: background_father_AUSTRIA

Feature adicionada: background_father_BRAZIL

Feature adicionada: background_father_CZECH

Feature adicionada: background_father_GERMANY

Feature adicionada: background_father_ISRAEL

Feature adicionada: background_father_ITALY

Feature adicionada: background_father_NETHERLANDS

Feature adicionada: background_father_POLAND

Feature adicionada: background_father_POMERANIA

Feature adicionada: background_father_PORTUGAL

Feature adicionada: background_father_SPAIN

Feature adicionada: background_father_UNK

Feature adicionada: background_mother_BRAZIL

Feature adicionada: background_mother_FRANCE

Feature adicionada: background_mother_GERMANY

Feature adicionada: ba

/home/wyctor/miniconda3/envs/lab-life/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wyctor/miniconda3/envs/lab-life/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wyctor/miniconda3/envs/lab-life/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

Feature adicionada: region_EAR

Feature adicionada: region_FACE

Feature adicionada: region_FOOT

Feature adicionada: region_FOREARM

Feature adicionada: region_HAND

Feature adicionada: region_LIP

Feature adicionada: region_NECK

Feature adicionada: region_NOSE

Feature adicionada: region_SCALP

Feature adicionada: region_THIGH

Feature adicionada: itch_False

Feature adicionada: itch_True

Feature adicionada: itch_UNK

Feature adicionada: grew_False

Feature adicionada: grew_True

Feature adicionada: grew_UNK

Feature adicionada: hurt_False

Feature adicionada: hurt_True

Feature adicionada: hurt_UNK

Feature adicionada: changed_False

Feature adicionada: changed_True

Feature adicionada: changed_UNK

Feature adicionada: bleed_False

Feature adicionada: bleed_True

Feature adicionada: bleed_UNK

Feature adicionada: elevation_False

Feature adicionada: elevation_True

Feature adicionada: elevation_UNK



In [357]:
save_results_to_csv(results, './results/naive-bayes/model_results_with_undersampling.csv')

# Uso do oversamplig para aumentar a quantidade de dados de cada classe e balancear as classes

In [358]:
from imblearn.over_sampling import RandomOverSampler

# Inicialize o RandomUnderSampler
ros = RandomOverSampler(random_state=42)

# Aplique a subamostragem
dataset_features_oversampled, y_encoded_oversampled = ros.fit_resample(dataset_features, y_encoded)

# Verifique o número de amostras por classe após o balanceamento
from collections import Counter
print("Distribuição original:", Counter(y_encoded))
print("Distribuição após subamostragem:", Counter(y_encoded_oversampled))


Distribuição original: Counter({np.int64(1): 845, np.int64(0): 273, np.int64(4): 192, np.int64(3): 72, np.int64(2): 52, np.int64(5): 40})
Distribuição após subamostragem: Counter({np.int64(1): 845, np.int64(0): 845, np.int64(4): 845, np.int64(5): 845, np.int64(3): 845, np.int64(2): 845})


In [359]:

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(dataset_features_oversampled, y_encoded_oversampled, test_size=0.2, random_state=42, shuffle=True)

In [360]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, roc_auc_score
from sklearn.naive_bayes import GaussianNB, BernoulliNB
# Modelo Naive Bayes
model = BernoulliNB()

# Lista para armazenar os resultados
results = []

# Incrementalmente incluir features
for i in range(1, X_train.shape[1]+1):
    print(f"Feature adicionada: {features[i-1]}\n")

    # Obter a quantidade de linhas de treino e teste
    width_train, _ = X_train.shape
    width_test, _ = X_test.shape
    
    # Selecionar as primeiras `i` features (e mantendo as linhas (amostras))
    X_train_subset = np.array(X_train.iloc[:, :i]).reshape(width_train, -1)
    X_test_subset = np.array(X_test.iloc[:, :i]).reshape(width_test, -1)
    
    # Treinar o modelo com as features selecionadas
    model.fit(X_train_subset, y_train)
    
    # Obter as probabilidades de saída para o conjunto de teste
    probas = model.predict_proba(X_test_subset)

    # Predições do modelo
    predictions = model.predict(X_test_subset)
    
    # Calcular métricas (roc_auc_score usa as probabilidades da classe positiva)
    roc_auc = roc_auc_score(y_test, probas, multi_class='ovr', average='macro') if len(np.unique(y_test)) > 2 else roc_auc_score(y_test, probas[:, 1])

    # Armazenar resultados
    results.append({
        'iteration_nº': i,
        'features_used': str(features[i-1]),
        'probabilities': probas,
        'predictions': predictions,
        'true_labels': y_test,
        'accuracy_score': accuracy_score(y_test, predictions),
        'balanced_accuracy_score': balanced_accuracy_score(y_test, predictions),
        'f1_score': f1_score(y_test, predictions, average='weighted'), 
        'precision_score': precision_score(y_test, predictions, average='weighted'),
        'roc_auc_score': roc_auc
    })


Feature adicionada: age

Feature adicionada: fitspatrick

Feature adicionada: diameter_1

Feature adicionada: diameter_2

Feature adicionada: biopsed

Feature adicionada: smoke_False

Feature adicionada: smoke_True

Feature adicionada: drink_False

Feature adicionada: drink_True

Feature adicionada: background_father_AUSTRIA

Feature adicionada: background_father_BRAZIL

Feature adicionada: background_father_CZECH

Feature adicionada: background_father_GERMANY

Feature adicionada: background_father_ISRAEL

Feature adicionada: background_father_ITALY

Feature adicionada: background_father_NETHERLANDS

Feature adicionada: background_father_POLAND

Feature adicionada: background_father_POMERANIA

Feature adicionada: background_father_PORTUGAL

Feature adicionada: background_father_SPAIN

Feature adicionada: background_father_UNK

Feature adicionada: background_mother_BRAZIL

Feature adicionada: background_mother_FRANCE

Feature adicionada: background_mother_GERMANY

Feature adicionada: ba

/home/wyctor/miniconda3/envs/lab-life/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wyctor/miniconda3/envs/lab-life/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wyctor/miniconda3/envs/lab-life/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

Feature adicionada: gender_MALE

Feature adicionada: skin_cancer_history_False

Feature adicionada: skin_cancer_history_True

Feature adicionada: cancer_history_False

Feature adicionada: cancer_history_True

Feature adicionada: has_piped_water_False

Feature adicionada: has_piped_water_True

Feature adicionada: has_sewage_system_False

Feature adicionada: has_sewage_system_True

Feature adicionada: region_ABDOMEN

Feature adicionada: region_ARM

Feature adicionada: region_BACK

Feature adicionada: region_CHEST

Feature adicionada: region_EAR

Feature adicionada: region_FACE

Feature adicionada: region_FOOT

Feature adicionada: region_FOREARM

Feature adicionada: region_HAND

Feature adicionada: region_LIP

Feature adicionada: region_NECK

Feature adicionada: region_NOSE

Feature adicionada: region_SCALP

Feature adicionada: region_THIGH

Feature adicionada: itch_False

Feature adicionada: itch_True

Feature adicionada: itch_UNK

Feature adicionada: grew_False

Feature adicionada: grew

In [361]:
save_results_to_csv(results, './results/naive-bayes/model_results_with_randomoversampling.csv')

Uso do SMOTE no oversampling

In [362]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler

# Inicialize o RandomUnderSampler
smote = SMOTE(random_state=0)

# Aplique a subamostragem
dataset_features_oversampled, y_encoded_oversampled = smote.fit_resample(dataset_features, y_encoded)

# Verifique o número de amostras por classe após o balanceamento
from collections import Counter
print("Distribuição original:", Counter(y_encoded))
print("Distribuição após subamostragem:", Counter(y_encoded_oversampled))


Distribuição original: Counter({np.int64(1): 845, np.int64(0): 273, np.int64(4): 192, np.int64(3): 72, np.int64(2): 52, np.int64(5): 40})
Distribuição após subamostragem: Counter({np.int64(1): 845, np.int64(0): 845, np.int64(4): 845, np.int64(5): 845, np.int64(3): 845, np.int64(2): 845})


In [363]:

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(dataset_features_oversampled, y_encoded_oversampled, test_size=0.2, random_state=42, shuffle=True)

In [364]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, roc_auc_score
from sklearn.naive_bayes import GaussianNB, BernoulliNB
# Modelo Naive Bayes
model = BernoulliNB()

# Lista para armazenar os resultados
results = []

# Incrementalmente incluir features
for i in range(1, X_train.shape[1]+1):
    print(f"Feature adicionada: {features[i-1]}\n")

    # Obter a quantidade de linhas de treino e teste
    width_train, _ = X_train.shape
    width_test, _ = X_test.shape
    
    # Selecionar as primeiras `i` features (e mantendo as linhas (amostras))
    X_train_subset = np.array(X_train.iloc[:, :i]).reshape(width_train, -1)
    X_test_subset = np.array(X_test.iloc[:, :i]).reshape(width_test, -1)
    
    # Treinar o modelo com as features selecionadas
    model.fit(X_train_subset, y_train)
    
    # Obter as probabilidades de saída para o conjunto de teste
    probas = model.predict_proba(X_test_subset)

    # Predições do modelo
    predictions = model.predict(X_test_subset)
    
    # Calcular métricas (roc_auc_score usa as probabilidades da classe positiva)
    roc_auc = roc_auc_score(y_test, probas, multi_class='ovr', average='macro') if len(np.unique(y_test)) > 2 else roc_auc_score(y_test, probas[:, 1])

    # Armazenar resultados
    results.append({
        'iteration_nº': i,
        'features_used': str(features[i-1]),
        'probabilities': probas,
        'predictions': predictions,
        'true_labels': y_test,
        'accuracy_score': accuracy_score(y_test, predictions),
        'balanced_accuracy_score': balanced_accuracy_score(y_test, predictions),
        'f1_score': f1_score(y_test, predictions, average='weighted'), 
        'precision_score': precision_score(y_test, predictions, average='weighted'),
        'roc_auc_score': roc_auc
    })


Feature adicionada: age

Feature adicionada: fitspatrick

Feature adicionada: diameter_1

Feature adicionada: diameter_2

Feature adicionada: biopsed

Feature adicionada: smoke_False

Feature adicionada: smoke_True

Feature adicionada: drink_False

Feature adicionada: drink_True

Feature adicionada: background_father_AUSTRIA

Feature adicionada: background_father_BRAZIL

Feature adicionada: background_father_CZECH

Feature adicionada: background_father_GERMANY

Feature adicionada: background_father_ISRAEL

Feature adicionada: background_father_ITALY

Feature adicionada: background_father_NETHERLANDS

Feature adicionada: background_father_POLAND

Feature adicionada: background_father_POMERANIA

Feature adicionada: background_father_PORTUGAL

Feature adicionada: background_father_SPAIN

Feature adicionada: background_father_UNK

Feature adicionada: background_mother_BRAZIL

Feature adicionada: background_mother_FRANCE

Feature adicionada: background_mother_GERMANY

Feature adicionada: ba

/home/wyctor/miniconda3/envs/lab-life/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wyctor/miniconda3/envs/lab-life/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wyctor/miniconda3/envs/lab-life/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

Feature adicionada: background_mother_NETHERLANDS

Feature adicionada: background_mother_NORWAY

Feature adicionada: background_mother_POLAND

Feature adicionada: background_mother_POMERANIA

Feature adicionada: background_mother_PORTUGAL

Feature adicionada: background_mother_SPAIN

Feature adicionada: background_mother_UNK

Feature adicionada: pesticide_False

Feature adicionada: pesticide_True

Feature adicionada: gender_FEMALE

Feature adicionada: gender_MALE

Feature adicionada: skin_cancer_history_False

Feature adicionada: skin_cancer_history_True

Feature adicionada: cancer_history_False

Feature adicionada: cancer_history_True

Feature adicionada: has_piped_water_False

Feature adicionada: has_piped_water_True

Feature adicionada: has_sewage_system_False

Feature adicionada: has_sewage_system_True

Feature adicionada: region_ABDOMEN

Feature adicionada: region_ARM

Feature adicionada: region_BACK

Feature adicionada: region_CHEST

Feature adicionada: region_EAR

Feature adicio

In [365]:
save_results_to_csv(results, './results/naive-bayes/model_results_with_smoteoversampling.csv')

Uso do oversampling via ADASYN

In [366]:
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import RandomOverSampler

# Inicialize o RandomUnderSampler
adasyn = ADASYN(random_state=0)

# Aplique a subamostragem
dataset_features_oversampled, y_encoded_oversampled = adasyn.fit_resample(dataset_features, y_encoded)

# Verifique o número de amostras por classe após o balanceamento
from collections import Counter
print("Distribuição original:", Counter(y_encoded))
print("Distribuição após subamostragem:", Counter(y_encoded_oversampled))


Distribuição original: Counter({np.int64(1): 845, np.int64(0): 273, np.int64(4): 192, np.int64(3): 72, np.int64(2): 52, np.int64(5): 40})
Distribuição após subamostragem: Counter({np.int64(4): 886, np.int64(2): 854, np.int64(1): 845, np.int64(5): 844, np.int64(3): 843, np.int64(0): 838})


In [367]:

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(dataset_features_oversampled, y_encoded_oversampled, test_size=0.2, random_state=42, shuffle=True)

In [368]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, roc_auc_score
from sklearn.naive_bayes import GaussianNB, BernoulliNB
# Modelo Naive Bayes
model = BernoulliNB()

# Lista para armazenar os resultados
results = []

# Incrementalmente incluir features
for i in range(1, X_train.shape[1]+1):
    print(f"Feature adicionada: {features[i-1]}\n")

    # Obter a quantidade de linhas de treino e teste
    width_train, _ = X_train.shape
    width_test, _ = X_test.shape
    
    # Selecionar as primeiras `i` features (e mantendo as linhas (amostras))
    X_train_subset = np.array(X_train.iloc[:, :i]).reshape(width_train, -1)
    X_test_subset = np.array(X_test.iloc[:, :i]).reshape(width_test, -1)
    
    # Treinar o modelo com as features selecionadas
    model.fit(X_train_subset, y_train)
    
    # Obter as probabilidades de saída para o conjunto de teste
    probas = model.predict_proba(X_test_subset)

    # Predições do modelo
    predictions = model.predict(X_test_subset)
    
    # Calcular métricas (roc_auc_score usa as probabilidades da classe positiva)
    roc_auc = roc_auc_score(y_test, probas, multi_class='ovr', average='macro') if len(np.unique(y_test)) > 2 else roc_auc_score(y_test, probas[:, 1])

    # Armazenar resultados
    results.append({
        'iteration_nº': i,
        'features_used': str(features[i-1]),
        'probabilities': probas,
        'predictions': predictions,
        'true_labels': y_test,
        'accuracy_score': accuracy_score(y_test, predictions),
        'balanced_accuracy_score': balanced_accuracy_score(y_test, predictions),
        'f1_score': f1_score(y_test, predictions, average='weighted'), 
        'precision_score': precision_score(y_test, predictions, average='weighted'),
        'roc_auc_score': roc_auc
    })


Feature adicionada: age

Feature adicionada: fitspatrick

Feature adicionada: diameter_1

Feature adicionada: diameter_2

Feature adicionada: biopsed

Feature adicionada: smoke_False

Feature adicionada: smoke_True

Feature adicionada: drink_False

Feature adicionada: drink_True

Feature adicionada: background_father_AUSTRIA

Feature adicionada: background_father_BRAZIL

Feature adicionada: background_father_CZECH

Feature adicionada: background_father_GERMANY

Feature adicionada: background_father_ISRAEL

Feature adicionada: background_father_ITALY

Feature adicionada: background_father_NETHERLANDS

Feature adicionada: background_father_POLAND

Feature adicionada: background_father_POMERANIA

Feature adicionada: background_father_PORTUGAL

Feature adicionada: background_father_SPAIN

Feature adicionada: background_father_UNK

Feature adicionada: background_mother_BRAZIL

Feature adicionada: background_mother_FRANCE

Feature adicionada: background_mother_GERMANY

Feature adicionada: ba

/home/wyctor/miniconda3/envs/lab-life/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wyctor/miniconda3/envs/lab-life/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wyctor/miniconda3/envs/lab-life/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

Feature adicionada: gender_FEMALE

Feature adicionada: gender_MALE

Feature adicionada: skin_cancer_history_False

Feature adicionada: skin_cancer_history_True

Feature adicionada: cancer_history_False

Feature adicionada: cancer_history_True

Feature adicionada: has_piped_water_False

Feature adicionada: has_piped_water_True

Feature adicionada: has_sewage_system_False

Feature adicionada: has_sewage_system_True

Feature adicionada: region_ABDOMEN

Feature adicionada: region_ARM

Feature adicionada: region_BACK

Feature adicionada: region_CHEST

Feature adicionada: region_EAR

Feature adicionada: region_FACE

Feature adicionada: region_FOOT

Feature adicionada: region_FOREARM

Feature adicionada: region_HAND

Feature adicionada: region_LIP

Feature adicionada: region_NECK

Feature adicionada: region_NOSE

Feature adicionada: region_SCALP

Feature adicionada: region_THIGH

Feature adicionada: itch_False

Feature adicionada: itch_True

Feature adicionada: itch_UNK

Feature adicionada: g

In [369]:
save_results_to_csv(results, './results/naive-bayes/model_results_with_adasyncmoversampling.csv')